# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = pd.read_csv("../output_data/cities.csv")
file.dropna()
file.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
temp_file=file.loc[(file['Max Temp']>=70)&(file['Max Temp']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
         # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -18.0520217,
                    "lng": 13.8400111
                },
                "viewport": {
                    "northeast": {
                        "lat": -18.05056242010728,
                        "lng": 13.84140747989272
                    },
                    "southwest": {
                        "lat": -18.05326207989273,
                        "lng": 13.83870782010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Opuwo Country Lodge",
            "photos": [
                {
                    "height": 1

{
    "html_attributions": [],
    "next_page_token": "Aap_uEBadAh4ykEpbjmPD8omGdHWkORFchcmhBvLpTvmzMURqvw27iYLl81bUIKf2qb1h_d4LzKsfhEzZU8Waq97Co9jYd0onVeNlM-zGlhu7kfI3R9961fLB3NuNSWUNHYsYWwrQgmynlvFzxoWGu-JqhXzpp1VaUK20tIwKUahBgvYD3WWM3JDtrRy-Nprsugjjl-dzZjNG8Q7e_CE13ajsQMYEyREEcBQ7UB2Q9pwtIhEUt-8jLypokrsIFY3ZB7x_WCwpY1AvfFj9yIkLIyiRVoClXE6XcO2iAxrxj-jbcw28chviPgOJg1FEiWbkFCIIQeBkcjgs7l_Vl6rLssNO56DNBR_Q_XByuoPZurR4h-k5LBVVkCrv_MZb3ryB1hFW_ZPa4UBBvkV7AhkJVQkWFKLjGowlRdRcD0eJrDQVZeKWRbJB3WAaHdmVcf5pB0F",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6441728,
                    "lng": -46.1058822
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.64274842010728,
                        "lng": -46.10453982010728
                    },
                    "southwest": {
                        "lat": -20.6454

{
    "html_attributions": [],
    "next_page_token": "Aap_uED_5kKyGnfIUmzAoqnO_BUp4W2V1FKPZwIU6mRWzXJQKkk8HV2M_kEu603APtjqgcEBPazkzlJkFwkNCt87hLoNqIgjBUca7v2gl97udApNw-qo7NGZ9oGCOjrvgr5o2dJ2QEylV3RMwuUOpaN1kHcax0omsvNi55RSaJoGlX8CpDUctsYa-AkZT7LdIsYXLq59atZmz-fSc1XcFb3TWx7XrUs4R6gSgPBuvGY-z8Rh4FT3yhPgDiUx7aokMkrTHaaMXDXsfhtM4XVlUBcuCw70a2Z4xZCCGXt_kJ55uCA5Fs9mye2o8qw2N99uIRQpbDZWD3GXt8nj8S_5NVtUx_oiFclkA-PuS8PKPSgOuZZ7eLVLF8NFjO8Tnm6PMkErrlP0Tk7W2TLQiDc8inbGws0ZFLRWCUPmUoujRenlDhromk5JBQkMxU_fqIWcImdD",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.7954115,
                    "lng": -50.0369189
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.79399982010728,
                        "lng": -50.03572867010728
                    },
                    "southwest": {
                        "lat": -29.7966

{
    "html_attributions": [],
    "next_page_token": "Aap_uEDXO0QD6pO5YxgqgQSK6szSlkood-SxXr_XEZvQWWT3gkpsP7hPhhqvr_3bhahjlbXmDc9glcfkD0hYYZp12eYoPdcxIq6lNjFr2g9B4NZyjmwo3v25acSxQ1lEAsJrnUGm_Qsd33o1Xkpzst1in3FWVo-98thEBLKCweLYkeO0bgCDTq0rKQebF5HYr43soKJCi-JsIlm-b1pRJHzlgmYnuvgfbYrExLCo3zNC9ck4TTSQ4aCyHhI2QMMF4MDVDSp8kC8f7xHM8O1lgC4H_N7o8LeMDC_zrrccKc3SYGGmq7Knxoc98_5Bz3rZU8WgmKA_Cw7Ag_AXHvXq0SNP6NW8Ya2JDsdjGzCY7LpoSeF2nTAmm2mDXCZ8PcipBq7s-6yRN9nhhaUDsZfJXOalghuFSn3gSLkpzGTLhXh-cVbPf7Om8Yjv_bXOf_9VlneB",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.3000259,
                    "lng": 44.2679367
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.29871172010727,
                        "lng": 44.26928077989272
                    },
                    "southwest": {
                        "lat": -20.301411

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 15.0020676,
                    "lng": 74.029939
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.00339517989272,
                        "lng": 74.0312313298927
                    },
                    "southwest": {
                        "lat": 15.00069552010728,
                        "lng": 74.02853167010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "La la land Resort",
            "photos": [
                {
                    "height": 4024,
  

{
    "html_attributions": [],
    "next_page_token": "Aap_uECDmtG1RIREqPDNHIWONyW081HaF81GolBgbAjBCZgB5q3Q0jTW_OI6h3-w8Oo_725slro2IfSxlDSYFEaBe0XD3DPK57oZOTKcDxezWCyU3sl2GdHcGBtpQ1oKzsyOPHitQT7QdMqgSVmRrm7KiUYu0MAiN2WSxKpc9vMJcSBZADkK9xPHeN7j-c3cVh54-Lf327XNfQZ9UwpXLgPtzKcPFFIgCvDvt2W3J6w5fPez6l5FqgVTKtvuC6p9FBvMw2qs06XFHCMKa6XfZBOv68eUzqg60UAvb5TetKJxYW0hAojVz72f5LsffqZcU25JDr5Y8OUGNNF_BG0L24CIVAPzxb2wW_9ukXdu1f9ict1AmH6qpIS-8NihJUzRClamyrD0FIy53vRFjo9dmA9QJ4I2n358lOY79K6ezSKdxS3o7nzfszral7tWBScJiLmg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.178516,
                    "lng": -2.9219246
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.18002257989272,
                        "lng": -2.920701170107277
                    },
                    "southwest": {
                        "lat": 35.177322920

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 22.9142468,
                    "lng": 96.4490175
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.91553782989272,
                        "lng": 96.45037157989272
                    },
                    "southwest": {
                        "lat": 22.91283817010727,
                        "lng": 96.44767192010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Golden Butterfly Hotel",
            "photos": [
                {
                    "height": 2

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8859566,
                    "lng": 59.220002
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.88738412989272,
                        "lng": 59.22126852989272
                    },
                    "southwest": {
                        "lat": 32.88468447010727,
                        "lng": 59.21856887010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Sepehr Hotel",
            "photos": [
                {
                    "height": 3264,
      

-------End of Search-------


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
1,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Balneário do Lago Hotel
2,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Xangri-lá Praia Hotel
3,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
4,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,La la land Resort
5,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Mercure Rif Nador
6,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Golden Butterfly Hotel
7,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Sepehr Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig




Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Observations for heat maps
# Ideal conditions a) A max temperature lower than 80 degrees but higher than 70, b) Wind speed less than 10 mph, c)Zero cloudiness.
#1 All but one of the hotels with the ideal locations seem to be located in coastal reagions.
#2 Sepehr Hotel located in Birjand Iran is the only hotel found inland with ideal conditions.


